## Callin Switzer
### 3 March 2017

1. Read in flower images
2. Segment images to remove background and centers 
3. Analyze color (using hue, saturation, and intensity)


In [2]:
import cv2

In [1]:
import numpy as np
import matplotlib
matplotlib.use("TkAgg") # have to use this for tkinter to  work below
from matplotlib import pyplot as plt
%matplotlib tk

# scikit image
import skimage
from skimage import io

import time

import os
import pandas as pd

/Users/callinswitzer/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [3]:
%qtconsole

In [4]:
# list files in directory

In [3]:
os.chdir('/Volumes/WeldShare/Hopkins Lab/Federico/Photos/Natural Populations 2015/')

In [4]:
# list files in directory
mypath = os.getcwd()
onlyfiles = [f for f in os.listdir(mypath) if os.path.isfile(os.path.join(mypath, f))]

In [7]:
# find bad contours
def is_contour_good(c, img):
    # approximate the contour
    area1 = cv2.contourArea(c, False) 

    # the contour is 'bad' if it is really small (less then 3000 px)
    # the contour is bad, if it is on the border
    # the contour is bad if it's really big
    x,y,w,h = cv2.boundingRect(c)       
    if x <= 1 or y <=1 or x+w+1 >= img.shape[1] or y+h+1 >= img.shape[0] or area1 < 90000 or area1 > 1000**2:
        return False              
    else:
        return True 

In [5]:
photoList = [ x for x in onlyfiles if not x.startswith('.') and x.endswith('.jpg')]

In [6]:
plants = [v.translate(None, '.jpg') for v in photoList]

In [115]:
# notes: 
# 1. This segmentation could be tuned to do better for light-colored plants
# 2. Somtimes the buds were also selected -- may want to calculate long axis and short axis to 
# reduce this problem

# plt.clf()
for kk in range(len(plants)):
    fpth = photoList[kk]
    img_color = cv2.imread(fpth)

    img = cv2.cvtColor(img_color, cv2.COLOR_BGR2GRAY)

    # plt.clf()
    # io.imshow(img) # so fast, compared to sklearn

    blur = cv2.GaussianBlur(img,(5,5),0)
    ret,th3 = cv2.threshold(blur,150,255,cv2.THRESH_BINARY_INV)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    erosion = cv2.erode(th3,kernel,iterations = 10)
    #io.imshow(erosion)

    dilation = cv2.dilate(erosion, kernel, iterations = 10)
    #plt.imshow(dilation, cmap = 'gray')

    # plt.xlim(6000, 0)
    # plt.ylim(0, 4000)
    # io.imshow(dilation)

    # close small holes
    image = dilation.copy()
    # plt.clf()
    # io.imshow(dilation)

    (cnts, _) = cv2.findContours(image.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    mask = np.ones(image.shape[:2], dtype="uint8") * 0 # create a blank black mask

    # loop over the contours
    for c in cnts:
        # if the contour is good, draw it on the mask (draw white on a black background)
        if is_contour_good(c, image):
            cv2.drawContours(mask, [c], -1, 255, -1)

    # see if any bad contours were found
    filHoles = 'no holes were filled--' + str((mask == image).all()) # will be false if a small hole was filled

    # remove the contours from the image and show the resulting images
    image2 = mask.copy()
    # plt.clf()
    # io.imshow( mask) # mask

    #io.imshow( image) # after

    # find contours for fruits
    cts, im2 = cv2.findContours(image2.copy(),  cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

    # get centers (moments)
    # compute the center of the contour
    Moms = np.array([cv2.moments(cc) for cc in cts])
    # get xy coordinates
    cs = np.array([[int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"])] for M in Moms])

    # calculate areas
    areas = [cv2.contourArea(cts[ii], False) for ii in range(len(cts))]


    # draw circle in middle of flower to exclude center 
    [cv2.circle(mask, tuple(cs[jj]), int(areas[jj]**(0.4)), (0,0,255), -1) for jj in range(len(areas))]
    # plt.clf()
    # io.imshow(mask)

    # apply mask to original image
    rgb = img_color[...,::-1] # convert to rgb, from bgr
    # plt.clf()
    # io.imshow(rgb)

    # apply mask to image
    res = cv2.bitwise_and(rgb,rgb,mask = mask)

    # io.imshow(res)
    saveDir = '/Volumes/WeldShare/Hopkins Lab/Federico/Photos/SegmentedImagesNaturalPopulations/'
    cv2.imwrite(saveDir +'segmented_' +  str(plants[kk]) + '.png', res)
    print str(kk + 1) + "_of_" + str(len(plants))

1_of_188
2_of_188
3_of_188
4_of_188
5_of_188
6_of_188
7_of_188
8_of_188
9_of_188
10_of_188
11_of_188
12_of_188
13_of_188
14_of_188
15_of_188
16_of_188
17_of_188
18_of_188
19_of_188
20_of_188
21_of_188
22_of_188
23_of_188
24_of_188
25_of_188
26_of_188
27_of_188
28_of_188
29_of_188
30_of_188
31_of_188
32_of_188
33_of_188
34_of_188
35_of_188
36_of_188
37_of_188
38_of_188
39_of_188
40_of_188
41_of_188
42_of_188
43_of_188
44_of_188
45_of_188
46_of_188
47_of_188
48_of_188
49_of_188
50_of_188
51_of_188
52_of_188
53_of_188
54_of_188
55_of_188
56_of_188
57_of_188
58_of_188
59_of_188
60_of_188
61_of_188
62_of_188
63_of_188
64_of_188
65_of_188
66_of_188
67_of_188
68_of_188
69_of_188
70_of_188
71_of_188
72_of_188
73_of_188
74_of_188
75_of_188
76_of_188
77_of_188
78_of_188
79_of_188
80_of_188
81_of_188
82_of_188
83_of_188
84_of_188
85_of_188
86_of_188
87_of_188
88_of_188
89_of_188
90_of_188
91_of_188
92_of_188
93_of_188
94_of_188
95_of_188
96_of_188
97_of_188
98_of_188
99_of_188
100_of_188
101_of_1

## read in images, and get an estimate of hue, saturation, brightness

In [7]:
os.chdir('/Volumes/WeldShare/Hopkins Lab/Federico/Photos/SegmentedImagesNaturalPopulations/')
segPlants= ['segmented_' +  str(plants[kk]) + '.png' for kk in range(len(plants))]

In [ ]:
segPlants = sorted(segPlants)

In [19]:

kk = 2
img1 = cv2.imread(segPlants[kk])
cv2.imshow('image', img1)
cv2.namedWindow('image',cv2.WINDOW_NORMAL)
cv2.resizeWindow('image', 600,600)

cv2.waitKey(0)
cv2.destroyAllWindows()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/callinswitzer/anaconda/lib/python2.7/site-packages/IPython/core/ultratb.py", line 1132, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/Users/callinswitzer/anaconda/lib/python2.7/site-packages/IPython/core/ultratb.py", line 313, in wrapped
    return f(*args, **kwargs)
  File "/Users/callinswitzer/anaconda/lib/python2.7/site-packages/IPython/core/ultratb.py", line 358, in _fixed_getinnerframes
    records = fix_frame_records_filenames(inspect.getinnerframes(etb, context))
  File "/Users/callinswitzer/anaconda/lib/python2.7/inspect.py", line 1044, in getinnerframes
    framelist.append((tb.tb_frame,) + getframeinfo(tb, context))
  File "/Users/callinswitzer/anaconda/lib/python2.7/inspect.py", line 1004, in getframeinfo
    filename = getsourcefile(frame) or getfile(frame)
  File "/Users/callinswitzer/anaconda/lib/python2.7/inspect.py", line 454, in getsourcefile
    if hasattr(getmodul

IndexError: string index out of range

In [17]:
cv2.destroyAllWindows()

In [18]:
# convert to rgb
rgbImg = cv2.cvtColor(img1,cv2.COLOR_BGR2RGB)
io.imshow(rgbImg)

In [154]:
def getAvgRGB(colorChannel, rgbImg):

    RR = np.array(rgbImg[:, :, colorChannel])

    flatR = RR.flatten()
    flatNZ = flatR[np.nonzero(flatR)]
    return np.mean(flatNZ)

# return average R, G, B values for the pixels in the image (excluding black)
[getAvgRGB(ii, rgbImg) for ii in range(3)]

[152.3561256652435, 77.574660077071499, 177.38634304593367]

In [182]:
# calculate rgb for every image
# colList = []
# for kk in range(len(segPlants)):
#     img1 = cv2.imread(segPlants[kk])
#     rgbImg = cv2.cvtColor(img1,cv2.COLOR_BGR2RGB)
#     cols = [getAvgRGB(ii, rgbImg) for ii in range(3)]
#     colList.append(np.hstack([segPlants[kk], cols]))
#     print str(kk + 1) + "_of_" + str(len(segPlants))
    
    

1_of_188
2_of_188
3_of_188
4_of_188
5_of_188
6_of_188
7_of_188
8_of_188
9_of_188
10_of_188
11_of_188
12_of_188
13_of_188
14_of_188
15_of_188
16_of_188
17_of_188
18_of_188
19_of_188
20_of_188


/Users/callinswitzer/anaconda/lib/python2.7/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)


21_of_188
22_of_188
23_of_188
24_of_188
25_of_188
26_of_188
27_of_188
28_of_188
29_of_188
30_of_188
31_of_188
32_of_188
33_of_188
34_of_188
35_of_188
36_of_188
37_of_188
38_of_188
39_of_188
40_of_188
41_of_188
42_of_188
43_of_188
44_of_188
45_of_188
46_of_188
47_of_188
48_of_188
49_of_188
50_of_188
51_of_188
52_of_188
53_of_188
54_of_188
55_of_188
56_of_188
57_of_188
58_of_188
59_of_188
60_of_188
61_of_188
62_of_188
63_of_188
64_of_188
65_of_188
66_of_188
67_of_188
68_of_188
69_of_188
70_of_188
71_of_188
72_of_188
73_of_188
74_of_188
75_of_188
76_of_188
77_of_188
78_of_188
79_of_188
80_of_188
81_of_188
82_of_188
83_of_188
84_of_188
85_of_188
86_of_188
87_of_188
88_of_188
89_of_188
90_of_188
91_of_188
92_of_188
93_of_188
94_of_188
95_of_188
96_of_188
97_of_188
98_of_188
99_of_188
100_of_188
101_of_188
102_of_188
103_of_188
104_of_188
105_of_188
106_of_188
107_of_188
108_of_188
109_of_188
110_of_188
111_of_188
112_of_188
113_of_188
114_of_188
115_of_188
116_of_188
117_of_188
118_of_188
1

In [183]:
newDF = pd.DataFrame(colList)
newDF.columns = ["fileName", "avgred", "avggreen", "avgblue"]
newDF

newDF.to_csv("SegmentedRGBVals.csv", index = False)

In [239]:
import time
colList = []
for kk in range(3):
    img1 = cv2.imread(segPlants[kk])
    rgbImg = cv2.cvtColor(img1,cv2.COLOR_BGR2RGB)
    plt.clf()
    plt.title(kk)
    io.imshow(rgbImg)
    time.sleep(0.2)


In [184]:
# pca
from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [226]:
X = newDF.ix[:, 1:4]

# conver to numeric
X = X.apply(pd.to_numeric,  errors = 'coerce')

X = X.dropna()


In [234]:
y = np.ones(len(X))
target_names = np.ones(len(X))

pca = PCA(n_components=2)
X_r = pca.fit(X).transform(X)

lda = LinearDiscriminantAnalysis(n_components=4)
X_r2 = lda.fit(X, y).transform(X)

# Percentage of variance explained for each components
print('explained variance ratio (first two components): %s'
      % str(pca.explained_variance_ratio_))

plt.clf()
colors = ['navy', 'turquoise', 'darkorange']
lw = 2

plt.scatter(X_r[:, 0], X_r[:, 1], color=color, alpha=.8, lw=lw)
plt.title('PCA')


explained variance ratio (first two components): [ 0.9339354   0.05064606]


In [235]:
plt.scatter(X_r2, X_r2, alpha=.8)
plt.title('LDA')

plt.show()

In [96]:
# convert to HSV
hsv_img = cv2.cvtColor(res,cv2.COLOR_RGB2HSV)

In [97]:
hsv_img.shape

(6536, 5104, 3)

In [98]:
plt.clf()
plt.hist(hsv_img[:,:, 1])

([array([ 6536.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
             0.,     0.]),
  array([ 6536.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
             0.,     0.]),
  array([ 6536.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
             0.,     0.]),
  array([ 6536.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
             0.,     0.]),
  array([ 6536.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
             0.,     0.]),
  array([ 6536.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
             0.,     0.]),
  array([ 6536.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
             0.,     0.]),
  array([ 6536.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
             0.,     0.]),
  array([ 6536.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
             0.,     0.]),
  array([ 6536.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
             0.,     0.]),


In [40]:
perimeters = [cv2.arcLength(cts[ii], True) for ii in range(len(cts))]

# calculate areas
areas = [cv2.contourArea(cts[ii], False) for ii in range(len(cts))]

# calculate circularity
# circularity = (perimeters .^ 2) ./ (4 * pi * area)
# If this is closer to 1, then the object is more circle-like
circularity = [(x **2) / (4*3.14159*y) for x,y in zip(perimeters, areas)]


arrs = np.array([i < 2 for i in circularity]) # get only roughly circular objects
# circles will be close to 0


ctd = np.array(cts)
filt_contours = ctd[arrs] # will let me see if we miss any fruits




# # draw filtered contours (white)
plt.clf()
# plt.xlim(6000, 0)
# plt.ylim(0, 4000)
cv2.drawContours(img, filt_contours, -1, (255,255,0), -1) # note, this modifies the original image, "img"
io.imshow(img)

In [ ]:
def kalmSeg(kk):
    # read in image as greyscale
    #kk = 70
    #kk += 1
    fpth = photoList[kk]
    img = cv2.imread(fpth,0)

    # plt.clf()
    # plt.xlim(6000, 0)
    # plt.ylim(0, 4000)
    # io.imshow(img) # so fast, compared to sklearn

    # Otsu's thresholding after Gaussian filtering
    blur = cv2.GaussianBlur(img,(5,5),0)
    # ret3,th3 = cv2.threshold(blur,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    ret,th3 = cv2.threshold(blur,125,255,cv2.THRESH_BINARY_INV)
    # I manually chose a threshold of 125 above



    # now get rid of the leafy bits
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    erosion = cv2.erode(th3,kernel,iterations = 10)
    #io.imshow(erosion)

    dilation = cv2.dilate(erosion, kernel, iterations = 10)
    #plt.imshow(dilation, cmap = 'gray')

    # plt.xlim(6000, 0)
    # plt.ylim(0, 4000)
    # io.imshow(dilation)

    # close small holes
    image = dilation.copy()
    #io.imshow(image)

    (cnts, _) = cv2.findContours(image.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    mask = np.ones(image.shape[:2], dtype="uint8") * 0 # create a blank black mask

    # loop over the contours
    for c in cnts:
        # if the contour is good, draw it on the mask (draw white on a black background)
        if is_contour_good(c, image):
            cv2.drawContours(mask, [c], -1, 255, -1)

    # see if any bad contours were found
    filHoles = 'no holes were filled--' + str((mask == image).all()) # will be false if a small hole was filled

    # remove the contours from the image and show the resulting images
    image2 = mask.copy()
    #io.imshow( mask) # mask

    #io.imshow( image) # after

    # find contours for fruits
    cts, im2 = cv2.findContours(image2.copy(),  cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

    perimeters = [cv2.arcLength(cts[ii], True) for ii in range(len(cts))]

    # calculate areas
    areas = [cv2.contourArea(cts[ii], False) for ii in range(len(cts))]

    # calculate circularity
    # circularity = (perimeters .^ 2) ./ (4 * pi * area)
    # If this is closer to 1, then the object is more circle-like
    circularity = [(x **2) / (4*3.14159*y) for x,y in zip(perimeters, areas)]


    arrs = np.array([i < 2 for i in circularity]) # get only roughly circular objects
    # circles will be close to 0


    ctd = np.array(cts)
    filt_contours = ctd[arrs] # will let me see if we miss any fruits




    # # draw filtered contours (white)
    # plt.clf()
    # plt.xlim(6000, 0)
    # plt.ylim(0, 4000)
    cv2.drawContours(img, filt_contours, -1, (255,255,0), -1) # note, this modifies the original image, "img"
    # io.imshow(img)

    # calculate diameters in pixels
    # area = pi * r ^2
    # r = sqrt(area / pi)

    radii = np.array([np.sqrt(aa / 3.14159) for aa in areas])

    # convert to mm instead of pixels
    dia_mm = np.array([aa2/max(radii) * 2 * 10 for aa2 in radii])


    # show circle with largest diameter (black)
    bgCirc = np.array(radii == radii.max())

    # draw filtered contours (black)
    cv2.drawContours(img, ctd[bgCirc], -1, (0,255,0), -1) # note, this modifies the original image, "img"
    #io.imshow(img)

    # compute the center of the contour
    Moms = np.array([cv2.moments(cc) for cc in cts])
    # get xy coordinates
    cs = np.array([[int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"])] for M in Moms])


    xx = np.array([cs[ii, 0] for ii in range(len(cs))])
    yy = np.array([cs[ii, 1] for ii in range(len(cs))])

    plt.clf()
    io.imshow(img)

    plt.plot(xx, yy, 'rp')
    plt.xlim(6000, 0)
    plt.ylim(0, 4000)

    # label diameters
    labels = ['dia_{0}'.format(round(i, 2)) for i in dia_mm]
    for label, x, y in zip(labels, xx, yy):
        plt.annotate(
            label, color = 'grey', size = 10,
            xy = (x, y), xytext = (0, 0),
            textcoords = 'offset points', ha = 'right', va = 'bottom')



    plt.show()

    # save plot, but make new directory if it doesn't already exist
    if not os.path.isdir("SegmentedImages/"):
        os.mkdir("SegmentedImages/")
        
    plt.savefig('SegmentedImages/' + str(plants[kk]) + '_segmented.png')

    print len(xx) # number of fruits (including big circle)
    print filHoles
    print 'kk = ' + str(kk)
    
    # save .csv file with information
    # return a pandas data frame
    tmp_DF = pd.DataFrame({'areas': areas, 
              'perimeters': perimeters, 
              'dia_mm': np.ndarray.tolist(dia_mm), 
             'plantNum': plants[kk]})
    return tmp_DF
    
    

In [ ]:
# create empty data frame for saving image data
df_final = pd.DataFrame({'areas': [], 
              'perimeters': [], 
              'dia_mm': [], 
             'plantNum': []})

#loop through all and save images
for yy in range(len(plants)):
    tmp_DF = kalmSeg(yy)
    
    #append to final data frame
    df_final = df_final.append(tmp_DF)
    

In [ ]:
# write the data to a file
df_final.to_csv('SegmentedImages/kalmiaFruitFinal.csv')

In [ ]:
plants[kk]

In [ ]:
df_final = pd.DataFrame({'areas': [], 
              'perimeters': [], 
              'dia_mm': [], 
             'plantNum': []})

tmp_DF = pd.DataFrame({'areas': areas, 
              'perimeters': perimeters, 
              'dia_mm': np.ndarray.tolist(dia_mm), 
             'plantNum': plants[kk]})


df_final = df_final.append(tmp_DF)

In [ ]:
# label diameters on plot

plt.clf()
io.imshow(img)

plt.plot(xx, yy, 'rp')
plt.xlim(6000, 0)
plt.ylim(0, 4000)

labels = ['dia_{0}'.format(round(i, 2)) for i in dia_mm]
for label, x, y in zip(labels, xx, yy):
    plt.annotate(
        label, color = 'grey', size = 10,
        xy = (x, y), xytext = (0, 0),
        textcoords = 'offset points', ha = 'right', va = 'bottom')
    
plt.show()

In [ ]:
# label points on plot

plt.clf()
io.imshow(img)

plt.plot(xx, yy, 'rp')
plt.xlim(6000, 0)
plt.ylim(0, 4000)

labels = ['point{0}'.format(i) for i in range(len(xx))]
for label, x, y in zip(labels, xx, yy):
    plt.annotate(
        label, color = 'grey', size = 10,
        xy = (x, y), xytext = (0, 0),
        textcoords = 'offset points', ha = 'right', va = 'bottom')
    
plt.show()